# SQL Subqueries - Lab

## Introduction

Now that you've seen how subqueries work, it's time to get some practice writing them! Not all of the queries will require subqueries, but all will be a bit more complex and require some thought and review about aggregates, grouping, ordering, filtering, joins and subqueries. Good luck!  

## Objectives

You will be able to:

* Write subqueries to decompose complex queries

## CRM Database Schema

Once again, here's the schema for the CRM database you'll continue to practice with.

<img src="images/Database-Schema.png" width="600">

## Connect to the Database

As usual, start by importing the necessary packages and connecting to the database **data.sqlite**.

In [2]:
#Your code here; import the necessary packages
import sqlite3 
conn = sqlite3.connect('data.sqlite')
c = conn.cursor()
import pandas as pd

In [3]:
#Your code here; create the connection and cursor

## Write an Equivalent Query using a Subquery

```SQL
select customerNumber,
       contactLastName,
       contactFirstName
       from customers
       join orders 
       using(customerNumber)
       where orderDate = '2003-01-31';
```

In [4]:
#Your code here; use a subquery. No join will be necessary.
c.execute("""select customerNumber, contactLastName, contactFirstName 
             from customers
             where customerNumber in (select customerNumber
                                   from orders
                                   where orderDate = '2003-01-31');""")
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df

,customerNumber,contactLastName,contactFirstName
0,141,Freyre,Diego


## Select the Total Number of Orders for Each Product Name

Sort the results by the total number of items sold for that product.

In [5]:
#Your code here
# c.execute("""select productName, quantityOrdered
#              from products
#              where productCode in (select productCode
#                                      from orderdetails
#                                      join products
#                                      using (productcode)
#                                      order by quantityOrdered desc);""")
c.execute("""select productName, quantityOrdered
             from products
             join orderdetails
             using (productCode)
             order by quantityOrdered desc""")
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df

,productName,quantityOrdered
0,1969 Dodge Charger,97
1,1969 Dodge Super Bee,90
2,America West Airlines B757-200,85
3,1970 Plymouth Hemi Cuda,77
4,F/A 18 Hornet 1/72,77
5,1917 Grand Touring Sedan,76
6,1949 Jaguar XK 120,76
7,1956 Porsche 356A Coupe,76
8,1962 Volkswagen Microbus,70
9,1956 Porsche 356A Coupe,70


## Select the Product Name and the  Total Number of People Who Have Ordered Each Product

Sort the results in descending order.

In [6]:
c.execute("""select productName,
                    count(distinct customerNumber) numPurchasers
                    from products
                    join orderdetails
                    using(productCode)
                    join orders
                    using (orderNumber)
                    group by productname
                    order by numPurchasers desc;""")
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df.head()

,productName,numPurchasers
0,1992 Ferrari 360 Spider red,40
1,1934 Ford V8 Coupe,27
2,1952 Alpine Renault 1300,27
3,1972 Alfa Romeo GTA,27
4,Boeing X-32A JSF,27


## Select the Employee Number, Office Code, City (of the office), and Name (First and Last) of those Employees who Sold Products that Have Been Ordered by Less Then 20 people.

This problem is a bit tougher. To start, think about how you might break the problem up. Be sure that your results only list each employee once.

In [7]:
c.execute("""select distinct employeeNumber,
                    officeCode,
                    o.city,
                    firstName,
                    lastName
                    from employees e
                    join offices o
                    using(officeCode)
                    join customers c
                    on e.employeeNumber = c.salesRepEmployeeNumber
                    join orders
                    using (customerNumber)
                    join orderdetails
                    using (orderNumber)
                    where productCode in (select productCode
                                                 from products
                                                 join orderdetails
                                                 using (productCode)
                                                 join orders
                                                 using (orderNumber)
                                                 group by productcode
                                                 having count(distinct customerNumber) <= 20
            );""")
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df

,employeeNumber,officeCode,city,firstName,lastName
0,1370,4,Paris,Gerard,Hernandez
1,1401,4,Paris,Pamela,Castillo
2,1621,5,Tokyo,Mami,Nishi
3,1501,7,London,Larry,Bott
4,1337,4,Paris,Loui,Bondur
5,1286,3,NYC,Foon Yue,Tseng
6,1166,1,San Francisco,Leslie,Thompson
7,1165,1,San Francisco,Leslie,Jennings
8,1612,6,Sydney,Peter,Marsh
9,1188,2,Boston,Julie,Firrelli


## Select the Employee Number, Name (First and Last) and Number of Customers of Employees Who's Customers Have an Average Credit Limit of Over 15K

In [13]:
#Your code here
# c.execute()
c.execute(""" SELECT employeeNumber, firstName, lastName, count(customerNumber)
              FROM employees e
              JOIN customers c
              ON e.employeeNumber = c.salesRepEmployeeNumber
              GROUP BY employeeNumber
              HAVING employeeNumber IN
                        (SELECT employeeNumber 
                         From employees e
                         JOIN customers c
                         ON e.employeeNumber = c.salesRepEmployeeNumber
                         GROUP BY employeeNumber
                         HAVING AVG(creditlimit) > 15000)""")
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df

,employeeNumber,firstName,lastName,count(customerNumber)
0,1165,Leslie,Jennings,6
1,1166,Leslie,Thompson,6
2,1188,Julie,Firrelli,6
3,1216,Steve,Patterson,6
4,1286,Foon Yue,Tseng,7
5,1323,George,Vanauf,8
6,1337,Loui,Bondur,6
7,1370,Gerard,Hernandez,7
8,1401,Pamela,Castillo,10
9,1501,Larry,Bott,8


## Summary

In this lesson, you got to practice some more complex SQL queries, some of which required subqueries. There's still plenty more SQL to be had though; hope you've been enjoying some of these puzzles!